<a href="https://colab.research.google.com/github/rishabhk0/end_to_end_ml/blob/Drowsiness_Detection_OpenCV/Agentic_AI_booking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your API Key: ")

Enter your API Key: ··········


In [ ]:
from decimal import MAX_PREC
import json
from openai import OpenAI

client = OpenAI()

""
#Salon Database


salons = [
    {
        "name" : "Berlin Style Cuts",
        "price" : 20,
        "rating" : 4.1,
        "available_slots" : ["Friday 18:00", "Saturday 14:00"]
    },
    {
        "name" : "Premium Barber Lounge",
        "price" : 25,
        "rating" : 4.6,
        "available_slots" : ["Monday 10:00", "Tuesday 16:00"]
    },
    {
        "name": "Berlin Hair Studio",
        "price": 18,
        "rating": 3.9,
        "available_slots": ["Wednesday 12:00", "Thursday 19:00"]
    }
]


def search_salons(max_price: int, day: str ):
    results = []

    for salon in salons:
        if salon["price"] <= max_price:
            for slot in salon["available_slots"]:
                if day.lower() in slot.lower() :
                   results.append(salon)

    if not results:
       return "No salons found matching criteria."

    results = sorted(results, key= lambda x: x["rating"], reverse =True)
    return results


def book_appointments(salon_name: str, slot: str):
    for salon in salons:
        if salon["name"]  == salon_name and slot in salon["available_slots"]:
           salon["available_slots"].remove(slot)
           return f" Appointment booked at {salon_name} on {slot}"
    return " slot not available"



tools = [
     {
         "type" : "function",
         "function" : {
             "name" : "search_salons",
             "description" : "Search for salons based on price and availability",
             "parameters" : {
                 "type" : "object",
                 "properties": {
                    "max_price": {"type": "integer"},
                     "day" : {"type":"string"}
                  },
                  "required" : ["max_price", "day"]
         }
     }
 },
     {
         "type" : "function",
         "function" : {
             "name" : "book_appointments",
             "description" : "Book an appointment at a salon",
             "parameters" : {
                 "type" : "object",
                 "properties": {
                     "salon_name" : {"type" : "string"},
                     "slot" : {"type" : "string"}
                 },
                 "required" : ["salon_name", "slot"]
             }
         }
     }

]



# Agent Function


def run_agent(user_input, messages=[]):
    messages.append({"role": "user","content": user_input})

    response = client.chat.completions.create(
         model = "gpt-4o-mini",
         messages=messages,
         tools = tools,
         tool_choice = "auto"
    )

    assistant_response_message = response.choices[0].message

    if assistant_response_message.tool_calls:
       messages.append(assistant_response_message)

       for tool_call_obj in assistant_response_message.tool_calls:
           function_name = tool_call_obj.function.name
           arguments = json.loads(tool_call_obj.function.arguments)

           result = None
           if function_name == "search_salons":
              result = search_salons(**arguments)
           elif function_name == "book_appointments":
                result = book_appointments(**arguments)
           else:
               result = f"Unknown function: {function_name}"

           messages.append({
               "role" : "tool",
               "tool_call_id" : tool_call_obj.id,
               "content" : str(result)
           })

       final_response = client.chat.completions.create(
            model = "gpt-4o-mini",
            messages = messages
        )

       final_message_content = final_response.choices[0].message.content
       messages.append({"role": "assistant", "content": final_message_content})

       return final_message_content
    else:
     messages.append({"role": "assistant", "content": assistant_response_message.content})
     return assistant_response_message.content


#Interactive loop

messages = [
    {"role": "system", "content": "you are an intelligent haircut booking assistant"}
]


while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "quit"]:
       break

    response = run_agent(user_input, messages)
    print("Agent", response)